# T4 · Ligand-based screening: compound similarity

Authors:

- Andrea Morger, 2017-2020, [Volkamer lab](https://volkamerlab.org/), Charité
- Franziska Fritz, CADD seminar, 2018, Charité/FU Berlin
- Yonghui Chen, 2019-2020, [Volkamer lab](https://volkamerlab.org/), Charité
- Dominique Sydow, 2018-2020, [Volkamer lab](https://volkamerlab.org/), Charité

Adapted by Gautier Peyrat

## Aim of this talktorial

In this talktorial, we get familiar with different approaches to **encode** (descriptors, fingerprints) and **compare** (similarity measures) molecules. Furthermore, we perform a **virtual screening** in the form of a similarity search for the EGFR inhibitor Gefitinib against our dataset of EGFR-tested molecules from the ChEMBL database filtered by Lipinski's rule of five (see **talktorial 2**). 

### Contents in _Theory_

* Molecular similarity
* Molecular descriptors
* Molecular fingerprints
  * Substructure-based fingerprints
  * MACCS fingerprints
  * Morgan fingerprints and circular fingerprints
* Molecular similarity measures
  * Tanimoto coefficient
  * Dice coefficient
* Virtual screening
  * Virtual screening using similarity search
  * Enrichment plots

### Contents in _Practical_

* Import and draw molecules
* Calculate molecular descriptors
  * 1D molecular descriptors: Molecular weight
  * 2D molecular descriptors: MACCS fingerprint
  * 2D molecular descriptors: Morgan fingerprint
* Calculate molecular similarity
  * MACCS fingerprints: Tanimoto and Dice similarity
  * Morgan fingerprints: Tanimoto and Dice similarity
* Virtual screening using similarity search
  * Compare query molecule to all molecules in the data set
  * Distribution of similarity values
  * Visualize most similar molecules
  * Generate enrichment plots
  * Calculate enrichment factors

### References

* Review on "Molecular similarity in medicinal chemistry" ([<i>J. Med. Chem.</i> (2014), <b>57</b>, 3186-3204](http://pubs.acs.org/doi/abs/10.1021/jm401411z))
* [Morgan fingerprints](http://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints) with `rdkit`
* Description of the extended-connectivity fingerprint ECFP ([<i>J. Chem. Inf. Model.</i> (2010), <b>50</b>,742-754](https://pubs.acs.org/doi/abs/10.1021/ci100050t))
* What is the chemical space?
([<i>ACS Chem. Neurosci.</i> (2012), <b>19</b>, 649-57](https://www.ncbi.nlm.nih.gov/pubmed/23019491))
* List of [molecular descriptors](https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors) in `rdkit`
* List of [fingerprints](https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-fingerprints) in `rdkit`
* Introduction to enrichment plots ([Applied Chemoinformatics, Wiley-VCH Verlag GmbH & Co. KGaA, Weinheim, (2018), **1**, 313-31](https://onlinelibrary.wiley.com/doi/10.1002/9783527806539.ch6h))

## Theory

### Molecular similarity 

Molecular similarity is a well known and often used concept in chemical informatics. Comparing molecules and their properties can be used in many different ways and help us to identify new molecules with desired properties and biological activity. 

The **similar property principle (SPP)** as well as the **structure-activity relationship (SAR)** approaches are based on the assumption that **structurally similar molecules** have **similar properties** and, thus, **similar biological activity**.  
In this context, **virtual screening** follows this idea to search for similar molecules in a given set of molecules with **known binding affinity** and to propose those as potentially new active molecules. 

### Molecular descriptors 

Similarity can be assessed in many different ways depending on the application (see [<i>J. Med. Chem.</i> (2014), <b>57</b>, 3186-3204</a>](http://pubs.acs.org/doi/abs/10.1021/jm401411z)): 

- 1D molecular descriptor: Solubility, logP, molecular weight, melting point.
  - **Global** descriptor: only one value represents the whole molecule
  - **Usually do not contain enough information** to be applied to machine learning (ML)
  - Can be added to 2D fingerprints to improve molecular encoding for ML
- 2D molecular descriptors: Molecular graphs, paths, fragments, atom environments
  - Detailed representation of individual parts of the molecule
  - Contains many features/bits per molecule called **fingerprints**
  - **Very often used** in similarity search and ML
- 3D molecular descriptors: Shape, stereochemistry
  - Less robust than 2D representations because of **molecule flexibility** (what is the "right" conformation of a molecule?)
    Biological similarity
  - Biological fingerprint, e.g. individual bits represent bioactivity measure against different targets
  - Independent of molecular structure
  - Requires experimental (or predicted) data

We already learned how to calculate 1D physicochemical parameters, such as the molecular weight and logP in **talktorial 2**.  
More information about these [descriptors](https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors) in the `rdkit` documentation.  

In the following, we focus on the definition of 2D molecular descriptors.  
Due to their, mostly, **uniqueness per molecule**, these descriptors are also called **fingerprints**.

### Molecular fingerprints

#### Substructure-based fingerprints

Molecular fingerprints are computational representations of molecules that **encode chemical and molecular features in the form of bitstrings, bitvectors or arrays**.  
Each bit corresponds to a predefined molecular feature or environment, where "1" represents the presence and "0" the absence of a feature.  
Note that some implementations are count-based. Thus, they count how often a specific feature is present.

There are **multiple ways** to design fingerprints.
Here, we introduce **MACCS keys** and **Morgan fingerprint** as two commonly used 2D fingerprints.  
As can be seen in the `rdkit` documentation on [fingerprints](https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-fingerprints), `rdkit` also offers multiple alternate fingerprints. 

#### MACCS fingerprints

Molecular ACCess System 
(MACCS) fingerprints, also termed MACCS structural keys, consist of **166 predefined structural fragments**. Each position queries the presence or absence of one particular structural fragment or key. 
The individual keys were empirically defined by medicinal chemists and are **simple to use and interpret** (`rdkit` documentation on [MACCS keys](http://rdkit.org/Python_Docs/rdkit.Chem.MACCSkeys-module.html)).

<img src="images/maccs_fp.png" alt="Figure 1" width="400"/>


*Figure 1*: Illustration of MACCS fingerprint (figure by Andrea Morger).

#### Morgan fingerprints and circular fingerprints 

This family of fingerprints is based on the **Morgan algorithm**. 
The bits correspond to the **circular environments** of each atom in a molecule. 
The number of neighboring bonds and atoms to consider is set by the **radius**. 
Also the length of the bit string can be defined, a longer bit string will be modded to the desired length. 
Therefore, the Morgan fingerprint is not limited to a certain number of bits. 
More about the [Morgan fingerprint](http://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints) can be found in the 
`rdkit` documentation. 
Extended connectivity fingerprints (ECFP) are also commonly used fingerprints that are derived using a **variant** of the Morgan algorithm, see [<i>J. Chem. Inf. Model.</i> (2010), <b>50</b>,742-754](https://pubs.acs.org/doi/abs/10.1021/ci100050t) for further information. 

<img src="images/morgan_fp.png" alt="Figure 2" width="400"/>

*Figure 2*: Illustration of the Morgan circular fingerprint (figure by Andrea Morger).

### Molecular similarity measures 

Once the descriptors/fingerprints are calculated, they can be compared to assess the similarity between two molecules. Molecular similarity can be quantified with a number of different similarity coefficients, two common similarity measures are the Tanimoto and Dice index ([<i>J. Med. Chem.</i> (2014), <b>57</b>, 3186-3204](http://pubs.acs.org/doi/abs/10.1021/jm401411z)).

#### Tanimoto coefficient

$$T _{c}(A,B) = \frac{c}{a+b-c}$$

- a: number of features present in molecule A 
- b: number of features present in molecule B 
- c: number of features shared by molecules A and B

#### Dice coefficient

$$D_{c}(A,B) = \frac{c}{\frac{1}{2}(a+b)}$$

- a: number of features present in molecule A
- b: number of features present in molecule B 
- c: number of features shared by molecules A and B

The similarity measures usually consider the number of positive bits (1's) present in either fingerprint and the number of positive bits that both have in common. 
Dice similarity usually returns higher values than Tanimoto similarity because of their denominators:

$$\frac{c}{a+b-c} \leq \frac{c}{\frac{1}{2}(a+b)}$$

### Virtual screening (VS)

The challenge in early stages of drug discovery is to **narrow down** a set of small molecules (molecules) from the large existing chemical space that are potentially binding to the target under investigation.  
Note that this chemical space is **vast**: Small molecules can be made of 10<sup>20</sup> combinations of chemical moieties ([<i>ACS Chem. Neurosci.</i> (2012), <b>19</b>, 649-57](https://www.ncbi.nlm.nih.gov/pubmed/23019491)). 

Since **experimental** high-throughput screening (HTS) for the activity of all those small molecules against the target of interest is **cost and time intensive**, computer-aided methods are invoked to propose a focused list of small molecules to be tested.  
This process is called **virtual (high-throughput) screening**: a large library of small molecules is filtered by rules and/or patterns, in order to identify those small molecules that are most likely to bind a target under investigation.

#### Virtual screening using similarity search

Comparing a set of novel molecules against a (or several) known active molecule(s) to find the most similar ones can be used as a simple way of virtual screening. 
Given the **similar property principle**, we can assume that the most similar molecules, e.g. to a known inhibitor, also have similar effects. Requirements for a similarity search are the following (as discussed in detail above):

* A **representation** that encodes chemical/molecular features
* A potential weighting of features (optional)
* A **similarity measurement**

A similarity search can be performed by calculating the similarity between one molecule and all molecules in a specific dataset. **Ranking** the molecules of the dataset by their similarity coefficient yields the most similar molecules at the top.

#### Enrichment plots

Enrichment plots are used to **validate virtual screening results**, which display the ratio of active molecules detected in the top x% of the ranked list, i.e.: 

* the ratio of top-ranked molecules (x-axis) from the whole dataset vs. 
* the ratio of active molecules (y-axis) from the whole dataset.

<img src="images/enrichment_plot.png" alt="Figure 3" width="500"/>

*Figure 3*: Example of enrichment plot for virtual screening results.

Remark:  
A good article about ROC and TOC for binary classification:  
https://towardsdatascience.com/roc-vs-toc-bccb7d70ef07

Here the enrichment plot is a TOC.

## Practical

In the first part of this practical section, we will use `rdkit` to **encode** molecules (molecular fingerprints) and **compare** them in order to calculate their similarity (molecular similarity measures), as discussed in the theory section above.

In the second part, we will use these encoding and comparison techniques to conduct a **similarity search (virtual screening)**: 
We use the known EGFR inhibitor Gefitinib as query and search for similar molecules in our data set of molecules tested on EGFR, which we collected from the ChEMBL database in **talktorial 1** and filtered by Lipinski's rule of five in **talktorial 2**.

### Import and draw molecules

First, we define and draw eight example molecules, which we will encode and compare later on.  
The molecules in SMILES format are converted to `rdkit` molecule objects and visualized with the `Draw` module in `rdkit`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from rdkit import Chem, DataStructs
from rdkit.Chem import (
    PandasTools,
    Draw,
    Descriptors,
    MACCSkeys,
    rdFingerprintGenerator,
)
from rdkit.Chem.Draw import IPythonConsole
from nbautoeval import run_yaml_quiz

import sys
sys.path.insert(1, f'../corrections/exercices')

from T4_mol_similarity import exo_calc_EF
from T4_mol_similarity import exo_calc_EF_opt

Let's define some molecule examples.

In [ ]:
# Molecules in SMILES format
molecule_smiles = [
    "CC1C2C(C3C(C(=O)C(=C(C3(C(=O)C2=C(C4=C1C=CC=C4O)O)O)O)C(=O)N)N(C)C)O", # doxycycline: antibiotic
    "CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=C(C=C3)O)N)C(=O)O)C", # amoxicilline: antibiotic
    "C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl", # furosemide: loop diuretic medication to treat edema
    "CCCCCCCCCCCC(=O)OCCOC(=O)CCCCCCCCCCC", # glycol_dilaurate:
    "C1NC2=CC(=C(C=C2S(=O)(=O)N1)S(=O)(=O)N)Cl", # hydrochlorothiazide: diuretic medication to treat hypertension
    "CC1=C(C(CCC1)(C)C)C=CC(=CC=CC(=CC(=O)O)C)C", # isotretinoine: treat severe acne
    "CC1(C2CC3C(C(=O)C(=C(C3(C(=O)C2=C(C4=C1C=CC=C4O)O)O)O)C(=O)N)N(C)C)O", # tetracycline: antibiotic
    "CC1C(CC(=O)C2=C1C=CC=C2O)C(=O)O", # hemi_cycline_D: (almost half of cyclines)
]

# List of molecule names
molecule_names = [
    "Doxycycline",
    "Amoxicilline",
    "Furosemide",
    "Glycol dilaurate",
    "Hydrochlorothiazide",
    "Isotretinoin",
    "Tetracycline",
    "Hemi-cycline D",
]

Generate a `DataFrame` for these molecules and add their ROMol objects.

In [ ]:
molecules = pd.DataFrame(# enter your code here)
molecules

In [ ]:
# Add ROMol column in the dataframe
# you have seen this method in previous notebooks

# Show first 2 molecules
molecules.head(2)

Visualize all molecules.

In [ ]:
# Draw molecules to a grid image with 3 molecules per row, and add arguments :
# subImgSize=(450,150), molsPerRow=3 and legends
Draw.MolsToGridImage(
    molecules["ROMol"].to_list(),
    molsPerRow=3,
    subImgSize=(450, 150),
    legends=molecules["name"].to_list(),
)

### Calculate molecular descriptors

We extract and generate 1D and 2D molecular descriptors to compare our molecules.  
For 2D descriptors, different types of fingerprints are generated to be used later for the calculation of the molecular similarity.

#### 1D molecular descriptors: Molecular weight

We calculate the molecular weight of our example structures.

In [ ]:
# Use pandas apply function to apply the MolWt function to all ROMol objects in the DataFrame
molecules["molecule_weight"] = ##

# Sort molecules in dataframe by molecular weight (descending order)
molecules.sort_values(["molecule_weight"], ascending=False, inplace=True)

We draw our molecular structures with their similar molecular weight for visual comparison: **Is the molecular weight a feasible descriptor for molecule similarity?**

In [ ]:
# Show only smiles, molecule names and molecular weights from the dataframe


We draw the molecules with their molecular weight.

In [ ]:
# Draw molecules to a grid image with 3 molecules per row, and add arguments :
# subImgSize=(450,150), molsPerRow=3 and
# legends=[f"{molecule['name']}: {molecule['molecule_weight']:.2f} Da" for index, molecule in molecules.iterrows()]
Draw.MolsToGridImage(
    molecules["ROMol"],
    legends=[
        f"{molecule['name']}: {molecule['molecule_weight']:.2f} Da"
        for index, molecule in molecules.iterrows()
    ],
    subImgSize=(450, 150),
    molsPerRow=3,
)

Question:  
What is your answer to the previous question: **Is the molecular weight a feasible descriptor for molecule similarity**?

In order to account for more detailed properties of a molecule, we now take a look at 2D molecular descriptors.

#### 2D molecular descriptors: MACCS fingerprint

__Example molecule__: Let's use an example molecule from our `DataFrame`.

In [ ]:
# select the first molecule of the dataframe "molecules"(Doxocycline)
molecule = 
molecule

In [ ]:
# select the second molecule of the dataframe "molecules"(Amoxicillin)
molecule2 = 
molecule2

MACCS fingerprints can be easily generated using `rdkit`.

In [ ]:
# Generate MACCS fingerprint for Doxocycline
maccs_fp1 = MACCSkeys.GenMACCSKeys(molecule)

In [ ]:
# Generate MACCS fingerprint for Amoxiciline

As explicit bitvectors are not human-readable, we will further transform them to bitstrings.

In [ ]:
# list all attributes and methods of the object maccs_fp1
maccs_fp1.

In [ ]:
# Print fingerprint maccs_fp1 as bitstring

__Applied to all molecules__: Add MACCS fingerprints for all molecules to the `DataFrame`.

In [ ]:
molecules["maccs"] = # Use the "apply" method on the "ROMol" column of the dataframe

#### 2D molecular descriptors: Morgan fingerprint

We can also calculate the circular Morgan fingerprints with `rdkit`.  
The Morgan fingerprint can be calculated either as int or bit vector. By default the radius is 2 and the vector is 2048 long. 

__Example molecule__: Let's use again with our example molecule.

Generate the Morgan fingerprint _as int vector_.

In [ ]:
circular_int_fp1 = rdFingerprintGenerator.GetCountFPs([molecule])[0]
circular_int_fp1

In [ ]:
# Generate Morgan fingerprint (as int vector) for the second molecule
circular_int_fp2 = 

In [ ]:
print(f"Print non-zero elements:\n{circular_int_fp1.GetNonzeroElements()}")

Generate the Morgan fingerprint _as bit vector_.

In [ ]:
# Note that the function takes a list as input parameter
# (even if we only want to pass one molecule)
circular_bit_fp1 = rdFingerprintGenerator.GetFPs([molecule])[0]
circular_bit_fp1

In [ ]:
# Generate Morgan fingerprint (as bit vector) for the second molecule
circular_bit_fp2 = 

In [ ]:
print(f"Print top 400 fingerprint bits:\n{circular_bit_fp1.ToBitString()[:400]}")

__Applied to all molecules__: Add Morgan fingerprints (as bit vector) for all molecules to the `DataFrame`.

In [ ]:
molecules["morgan"] = rdFingerprintGenerator.GetFPs(molecules["ROMol"].tolist())

### Calculate molecular similarity

In the following, we will apply two similarity measures, i.e. **Tanimoto** and **Dice**, to our two fingerprint types, i.e. **MACCS** and **Morgan** fingerprints.

__Example molecule pair__: Compare two MACCS fingerprints with the Tanimoto similarity.

In [ ]:
# Example molecules
molecule1 = molecules["ROMol"][0]
molecule2 = molecules["ROMol"][1]

# Example fingerprints
maccs_fp1 = MACCSkeys.GenMACCSKeys(molecule1)
maccs_fp2 = MACCSkeys.GenMACCSKeys(molecule2)

Calculate the Tanimoto coefficient _between two different molecules_.

In [ ]:
# list all methods and attributes from "DataStructs",
# and calculate Tanimoto simlarity btw 2 different molecules
DataStructs.

Calculate Tanimoto coefficient _between the same molecule_.

In the following, we want to compare a query molecule with our molecule list. 
Therefore, we use `BulkTanimotoSimilarity` and `BulkDiceSimilarity` from `rdkit` that calculate the similarity of a query fingerprint with a list of fingerprints, based on a similarity measure. We will investigate all combinations of MACCS/Morgan fingerprint comparisons based on the Tanimoto/Dice similarity measure.

#### MACCS fingerprints: Tanimoto similarity and Dice similarity

Add the Tanimoto and Dice similarities between MACCS fingerprints to the `DataFrame`.

In [ ]:
# Define molecule query and list
molecule_query = # Select the first MACCS of the dataframe
molecule_list = # Get all MACCS in a list

# Calculate similarty values between query and list elements
molecules["tanimoto_maccs"] = # BulkTanimotoSimilarity can be found in DataStructs
molecules["dice_maccs"] = # BulkDiceSimilarity can be found in DataStructs

Sort the `DataFrame` by the Tanimoto similarity of the MACCS fingerprints.

In [ ]:
preview = molecules.sort_values(["tanimoto_maccs"], ascending=False).reset_index()
# show only name and similarity columns ("name", "tanimoto_maccs", "dice_maccs")
preview[["name", "tanimoto_maccs", "dice_maccs"]]

__Note__: Tanimoto and Dice similarity measures by definition result in the same ranking of molecules, with higher values for the Dice similarity (see Tanimoto and Dice equations in the theory section of this talktorial).

Draw molecules ranked by the Tanimoto similarity of MACCS fingerprints.

In [ ]:
def draw_ranked_molecules(molecules, sort_by_column):
    """
    Draw molecules sorted by a given column.

    Parameters
    ----------
    molecules : pandas.DataFrame
        Molecules (with "ROMol" and "name" columns and a column to sort by.
    sort_by_column : str
        Name of the column used to sort the molecules by.

    Returns
    -------
    Draw.MolsToGridImage
        2D visualization of sorted molecules.
    """
    # sort the "molecules" dataframe depending on "sort_by_column" parameter
    molecules_sorted = molecules.sort_values([sort_by_column], ascending=False).reset_index()
    
    return Draw.MolsToGridImage(
        molecules_sorted["ROMol"],
        legends=[
            f"#{index+1} {molecule['name']}, similarity={molecule[sort_by_column]:.2f}"
            for index, molecule in molecules_sorted.iterrows()
        ],
        molsPerRow=3,
        subImgSize=(450, 150),
    )
    # Draw.MolsToGridImage with molsPerRow=3, subImgSize=(450,150) and legends:
    # legends=[f"#{index+1} {molecule['name']}, similarity={molecule[sort_by_column]:.2f}" 
    #          for index, molecule in molecules_sorted.iterrows()]

In [ ]:
# Draw ranked molecules by tanimoto_maccs

With MACCS fingerprints, Tetracycline is the most similar molecule (high score), followed by Amoxicilline. In contrast to the 1D descriptor molecular weight, the linear molecule Glycol dilaurate is recognized as dissimilar (last rank).

#### Morgan fingerprints: Tanimoto similarity and Dice similarity

Add the Tanimoto and Dice similarities between the Morgan fingerprints to the `DataFrame`.

In [ ]:
# Define molecule query and list
molecule_query = molecules["morgan"][0]
molecule_list = molecules["morgan"].to_list()

# Calculate similarty values between query and list elements
molecules["tanimoto_morgan"] = DataStructs.BulkTanimotoSimilarity(molecule_query, molecule_list)
molecules["dice_morgan"] = DataStructs.BulkDiceSimilarity(molecule_query, molecule_list)

Sort the `DataFrame` by the Tanimoto similarity of the Morgan fingerprints.

In [ ]:
preview = molecules.sort_values(["tanimoto_morgan"], ascending=False).reset_index()

# show "name", "tanimoto_morgan", "dice_morgan", "tanimoto_maccs" and "dice_maccs"
preview[["name", "tanimoto_morgan", "dice_morgan", "tanimoto_maccs", "dice_maccs"]]

Draw molecules ranked by the Tanimoto similarities of Morgan fingerprints.

In [ ]:
# Draw ranked molecules by tanimoto_morgan
draw_ranked_molecules(molecules, "tanimoto_morgan")

We compare the Tanimoto similarities based on MACCS and Morgan fingerprints. Therefore, we plot the Tanimoto similarities based on Morgen fingerprints vs. the Tanimoto simlarities based on MACCS fingerprints.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
molecules.plot("tanimoto_maccs", "tanimoto_morgan", kind="scatter", ax=ax)
ax.plot([0, 1], [0, 1], "k--")
ax.set_xlabel("Tanimoto (MACCS)")
ax.set_ylabel("Tanimoto (Morgan)")
fig;

Usage of different fingerprints (here: MACCS and Morgan fingerprints) results in different similarity values (here: Tanimoto similarity) and thus potentially also in different rankings of molecule similarity as shown here. 

Morgan fingerprints also recognize Tetracycline as the most similar molecule to Doxycycline (but with a lower score), and Clycol dilaurate as most dissimilar. However, ranked second is Hemi-cycline D, a structural part of cyclines - possibly because of the atom environment-based algorithm of Morgan fingerprints (whereas MACCS fingerprints rather ask for the occurrence of certain properties). 

### Virtual screening using similarity search

Now that we have learned how to calculate fingerprints and the similarity between them, we can apply this knowledge to a similarity search of a query molecule against a full data set of molecules. 

We use the known EGFR inhibitor Gefitinib as query and search for similar molecules in our data set of molecules tested on EGFR, which we collected from the ChEMBL database in **talktorial 1** and filtered by Lipinski's rule of five in **talktorial 2**.

#### Compare query molecule to all molecules in the data set

We import molecules from a *csv* file containing the filtered EGFR-tested molecules from the ChEMBL database as provided by **talktorial 2**. Given one query molecule (here Gefitinib), we screen that data set for similar molecules.

__Dataset__: Import data from the _csv_ file containing molecules in the SMILES format.

In [ ]:
molecule_dataset = pd.read_csv(
    "../data/T2/EGFR_compounds_lipinski.csv",
    delimiter=';',
    usecols=["molecule_chembl_id", "smiles", "pIC50"],
)
print(f"Number of molecules in dataset: {len(molecule_dataset)}")
molecule_dataset.head(5)

__Query__: For the query molecule Gefitinib, generate the `ROMol` object from the query molecule's SMILES.

In [ ]:
# You can find its SMILES on http://www.icoa.fr/pkidb/
query = # Molecule Gefitinib
query

Generate MACCS and Morgan fingerprints for the query molecule.

In [ ]:
maccs_fp_query = #
circular_fp_query = #

Generate MACCS and Morgan fingerprints for all molecules in our dataset.

In [ ]:
# Add Molecules column in the dataframe

# generate MACCS fp
maccs_fp_list = 

# generate Morgan fp
circular_fp_list = 

Calculate the Tanimoto similarity between the query molecule (Gefitinib) and all molecules our dataset (using MACCS and Morgan fingerprints).

In [ ]:
molecule_dataset["tanimoto_maccs"] = DataStructs.BulkTanimotoSimilarity(
    maccs_fp_query, maccs_fp_list
)
molecule_dataset["tanimoto_morgan"] = DataStructs.BulkTanimotoSimilarity(
    circular_fp_query, circular_fp_list
)

Calculate Dice similarity for query molecule (Gefitinib) and all molecules in file (MACCS, Morgan).

In [ ]:
molecule_dataset["dice_maccs"] = DataStructs.BulkDiceSimilarity(maccs_fp_query, maccs_fp_list)
molecule_dataset["dice_morgan"] = DataStructs.BulkDiceSimilarity(
    circular_fp_query, circular_fp_list
)

In [ ]:
# Select the columns "smiles", "tanimoto_maccs", "tanimoto_morgan", "dice_maccs", "dice_morgan" 
# Show DataFrame the 5 first lines of the dataframe
molecule_dataset[
    ["smiles", "tanimoto_maccs", "tanimoto_morgan", "dice_maccs", "dice_morgan"]
].head(5)

In [ ]:
# Show all columns for the 3 first lines of the dataframe
molecule_dataset.head(3)

#### Distribution of similarity values

As mentioned in the theory section, compared for the same fingerprint (e.g. MACCS fingerprints), Tanimoto similarity values are lower than Dice similarity values.
Also, comparing two different fingerprints (e.g. MACCS and Morgan fingerprints), the similarity measure values (e.g. Tanimotot similarity) vary. 

We can have a look at the distributions by plotting histograms.

First, we can plot distribution of Tanimoto similarity of MACCS fingerprints.

In [ ]:
fig, axes = plt.subplots(figsize=(10, 6), nrows=2, ncols=2)
molecule_dataset.hist(["tanimoto_maccs"], ax=axes[0, 0])
molecule_dataset.hist(["tanimoto_morgan"], ax=axes[0, 1])
molecule_dataset.hist(["dice_maccs"], ax=axes[1, 0])
molecule_dataset.hist(["dice_morgan"], ax=axes[1, 1])
axes[1, 0].set_xlabel("similarity value")
axes[1, 0].set_ylabel("# molecules")
fig;

Second, we can compare similarities here. This time, let's directly compare Tanimoto and Dice similarities for the two fingerprints.

In [ ]:
fig, axes = plt.subplots(figsize=(12, 6), nrows=1, ncols=2)

molecule_dataset.plot("tanimoto_maccs", "dice_maccs", kind="scatter", ax=axes[0])
axes[0].plot([0, 1], [0, 1], "k--")
axes[0].set_xlabel("Tanimoto (MACCS)")
axes[0].set_ylabel("Dice (MACCS)")

molecule_dataset.plot("tanimoto_morgan", "dice_morgan", kind="scatter", ax=axes[1])
axes[1].plot([0, 1], [0, 1], "k--")
axes[1].set_xlabel("Tanimoto (Morgan)")
axes[1].set_ylabel("Dice (Morgan)")

fig;

Similarity distributions are important to interpret similarity values, e.g. a value of 0.6 needs to be evaluated differently for MACCS or Morgan fingerprints, as well as Tanimoto or Dice similarities.

#### Visualize most similar molecules

We visually inspect the structure of Gefitinib in comparison to the most similar molecules in our ranking, including the information about their bioactivity (pIC50 derived from the ChEMBL database in **talktorial 1**).

Here, we sort the `DataFrame` by the Tanimoto similarity based on Morgan fingerprints.

In [ ]:
# sort by descending order, show the first 3 lines
molecule_dataset.sort_values(["tanimoto_morgan"], ascending=False).head(3)

Show the query and its most similar molecules alongside the molecules' bioactivities.

In [ ]:
top_n_molecules = 10
# sort dataframe by descending order, using tanimoto similarity and morgan fingerprints, and reset_index in the end
top_molecules = 

# select the first 10 rows using "top_n_molecules"
top_molecules = 

legends = [
    f"#{index+1} {molecule['molecule_chembl_id']}, pIC50={molecule['pIC50']:.2f}"
    for index, molecule in top_molecules.iterrows()
]

# Draw the molecules in a grid image
Chem.Draw.MolsToGridImage(
    mols=[query] + top_molecules["ROMol"].tolist(),
    legends=(["Gefitinib"] + legends),
    molsPerRow=3,
    subImgSize=(450, 150),
)

The top ranked molecules for Gefitinib are Gefitinib entries (rank 1 and 2) in our dataset, followed by alterations of Gefitinib, e.g. different benzole substitution patterns. 
Note: ChEMBL contains the complete structure-activity relationship analysis for Gefitinib (being a well-studied molecule), therefore it is not surprising to have that many Gefitinib-like molecules in our dataset.

We now check **how well the similarity search is able to distinguish between active and inactive molecules based on our dataset**. Therefore, we use the bioactivity values, which we collected from ChEMBL for each molecule (bioactivity against EGFR) in **talktorial 1**.

#### Generate enrichment plots

In order to validate our virtual screening and see the ratio of detected active molecules, we generate an enrichment plot. 

__Enrichment plots__ show 

* the ratio of top-ranked molecules (x-axis) from the whole dataset vs. 
* the ratio of active molecules (y-axis) from the whole dataset. 

We compare the Tanimoto similarity for MACCS and Morgan fingerprints. 

In order to decide whether we treat a molecule as active or inactive, we apply the commonly used pIC50 cutoff value of 6.3. Although there are several suggestions ranging from an pIC50 cutoff values of 5 to 7 in the literature or even to define an exclusion range, we think this cutoff is reasonable. 
The same cutoff will be used for machine learning in **talktorial 7**.

In [ ]:
molecule_dataset.head(3)

In [ ]:
def get_enrichment_data(molecules, similarity_measure, pic50_cutoff):
    """
    Calculates x and y values for enrichment plot:
        x - % ranked dataset
        y - % true actives identified

    Parameters
    ----------
    molecules : pandas.DataFrame
        Molecules with similarity values to a query molecule.
    similarity_measure : str
        Column name which will be used to sort the DataFrame．
    pic50_cutoff : float
        pIC50 cutoff value used to discriminate active and inactive molecules.

    Returns
    -------
    pandas.DataFrame
        Enrichment data: Percentage of ranked dataset by similarity vs. percentage of identified true actives.
    """

    # Get number of molecules in data set
    molecules_all = len(molecules)

    # Get number of active molecules in data set
    actives_all = sum(molecules["pIC50"] >= pic50_cutoff)

    # Initialize a list that will hold the counter for actives and molecules while iterating through our dataset
    actives_counter_list = []

    # Initialize counter for actives (to 0)
    actives_counter = 0

    # Note: Data must be ranked for enrichment plots:
    # Sort molecules by selected similarity measure

    # Iterate over the ranked dataset and check each molecule if active (by checking bioactivity)
    for value in molecules["pIC50"]:
        if value >= pic50_cutoff:
            actives_counter += 1
        actives_counter_list.append(actives_counter)

    # Transform number of molecules into % ranked dataset
    molecules_percentage_list = [i / molecules_all for i in range(1, molecules_all + 1)]

    # Transform number of actives into % true actives identified
    actives_percentage_list = [i / actives_all for i in actives_counter_list]

    # Generate DataFrame with x and y values as well as label
    enrichment = pd.DataFrame(
        {
            "% ranked dataset": molecules_percentage_list,
            "% true actives identified": actives_percentage_list,
        }
    )

    return enrichment

Set the pIC50 cutoff used to discriminate active and inactive molecules... 

In [ ]:
pic50_cutoff = 6.3

...and get the enrichment for MACCS and Morgan fingerprints (using the Tanimoto similarity).

In [ ]:
similarity_measures = ["tanimoto_maccs", "tanimoto_morgan"]
# Generate a dictionnary of DataFrames containing similarity_measure in keys,
# and enrichment_data for all similarity_measures
enrichment_data = {}

Show the first lines of enrichment data, by example for "tanimoto_maccs"

In [ ]:
#

Plot the enrichment data next to the optimal and random enrichment curve!

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

fontsize = 20

# Plot enrichment data
for similarity_measure, enrichment in enrichment_data.items():
    ax = enrichment.plot(
        ax=ax,
        x="% ranked dataset",
        y="% true actives identified",
        label=similarity_measure,
        alpha=0.5,
        linewidth=4,
    )
ax.set_ylabel("% True actives identified", size=fontsize)
ax.set_xlabel("% Ranked dataset", size=fontsize)

# Plot optimal curve: Ratio of actives in dataset
ratio_actives = sum(molecule_dataset["pIC50"] >= pic50_cutoff) / len(molecule_dataset)
ax.plot(
    [0, ratio_actives, 1],
    [0, 1, 1],
    label="Optimal curve",
    color="black",
    linestyle="--",
)

# Plot random curve
ax.plot([0, 1], [0, 1], label="Random curve", color="grey", linestyle="--")

plt.tick_params(labelsize=16)
plt.legend(
    labels=["MACCS", "Morgan", "Optimal", "Random"],
    loc=(0.5, 0.08),
    fontsize=fontsize,
    labelspacing=0.3,
)

# Save plot -- use bbox_inches to include text boxes
plt.savefig(
    "../data/T4/enrichment_plot.png",
    dpi=300,
    bbox_inches="tight",
    transparent=True,
)

plt.show()

The enrichment plots show a slightly better performance for fingerprint comparison based on Morgan fingerprints than based on MACCS fingerprints.

#### Calculate enrichment factors

The (experimental) __enrichment factor (EF)__ is used to access a methods success by a single number rather than a plot. The idea is to determine the percentage of active molecules in the top x% (e.g. 5%) of ranked molecules, as defined by the following helper function.

In [ ]:
exo_calc_EF.example()

In [ ]:
def calculate_enrichment_factor(enrichment, ranked_dataset_percentage_cutoff):
    """
    Get the experimental enrichment factor for a given percentage of the ranked dataset.

    Parameters
    ----------
    enrichment : pd.DataFrame
        Enrichment data: Percentage of ranked dataset by similarity vs. percentage of
        identified true actives.
    ranked_dataset_percentage_cutoff : float or int
        Percentage of ranked dataset to be included in enrichment factor calculation.

    Returns
    -------
    float
        Experimental enrichment factor.
    """

    # Keep only molecules that meet the cutoff from enrichment dataframe
    enrichment = 
    # Get highest percentage of actives and the corresponding percentage of true actives identified
    highest_enrichment = enrichment.iloc[-1]
    # calculate enrichment factor, EF = 100* highest percentage of true actives identified
    enrichment_factor = round(100 * float(highest_enrichment["% true actives identified"]), 1)
    return 

In [ ]:
exo_calc_EF.correction(calculate_enrichment_factor)

This helper function calculates the random enrichment of active molecules in the top x% of ranked molecules (__random enrichment factor__).

In [ ]:
def calculate_enrichment_factor_random(ranked_dataset_percentage_cutoff):
    """
    Get the random enrichment factor for a given percentage of the ranked dataset.

    Parameters
    ----------
    ranked_dataset_percentage_cutoff : float or int
        Percentage of ranked dataset to be included in enrichment factor calculation.

    Returns
    -------
    float
        Random enrichment factor.
    """
    # convert the value to a float having 1 decimal place
    enrichment_factor_random = round(float(ranked_dataset_percentage_cutoff), 1)
    return enrichment_factor_random

And this helper function calculates the optimal enrichment of active molecules in the top x% of ranked molecules (__optimal enrichment factor__). The random and optimal enrichment factors are needed to judge how good the experimental enrichment factor for a method is.

In [ ]:
exo_calc_EF_opt.example()

In [ ]:
def calculate_enrichment_factor_optimal(molecules, ranked_dataset_percentage_cutoff, pic50_cutoff):
    """
    Get the optimal random enrichment factor for a given percentage of the ranked dataset.

    Parameters
    ----------
    molecules : pandas.DataFrame
        the DataFrame with all the molecules and pIC50.
    ranked_dataset_percentage_cutoff : float or int
        Percentage of ranked dataset to be included in enrichment factor calculation.
    activity_cutoff: float
        pIC50 cutoff value used to discriminate active and inactive molecules

    Returns
    -------
    float
        Optimal enrichment factor.
    """
    # Calculate the percentage of active molecules in the dataset
    ratio = 
    # Add a condition to compare the ranked_dataset_percentage_cutoff and the ratio, if cutoff <= ratio, 
    # enrichment_factor_optimal = 100 / ratio * ranked_dataset_percentage_cutoff, else enrichment_factor_optimal = 100.0
    # round the results to one decimal place
    if ranked_dataset_percentage_cutoff <= ratio:
        enrichment_factor_optimal = round(100 / ratio * ranked_dataset_percentage_cutoff, 1)
    else:
        enrichment_factor_optimal = 100.0
    return enrichment_factor_optimal

In [ ]:
exo_calc_EF_opt.correction(calculate_enrichment_factor_optimal)

We are ready to calculate the experimental enrichment factor (EF) for 5% of ranked dataset... 

In [ ]:
ranked_dataset_percentage_cutoff = 5

In [ ]:
for similarity_measure, enrichment in enrichment_data.items():
    enrichment_factor = calculate_enrichment_factor(enrichment, ranked_dataset_percentage_cutoff)
    print(
        f"Experimental EF for {ranked_dataset_percentage_cutoff}% of ranked dataset ({similarity_measure}): {enrichment_factor}%"
    )

... and we can compare it to the random and optimal enrichment factor.

In [ ]:
enrichment_factor_random = calculate_enrichment_factor_random(ranked_dataset_percentage_cutoff)
print(
    f"Random EF for {ranked_dataset_percentage_cutoff}% of ranked dataset: {enrichment_factor_random}%"
)
enrichment_factor_optimal = calculate_enrichment_factor_optimal(
    molecule_dataset, ranked_dataset_percentage_cutoff, pic50_cutoff
)
print(
    f"Optimal EF for {ranked_dataset_percentage_cutoff}% of ranked dataset: {enrichment_factor_optimal}%"
)

## Discussion

We have learned how to use fingerprints and similarity measures to compare a query molecule with a dataset of molecules and rank the molecule by similarity. This approach is an example for virtual screening of molecules.
Enrichment plots and enrichment factors can be used to assess the performance. 

A **drawback** of a similarity search with molecular fingerprints is that it is based on molecular similarity and thus **does not yield any novel structures**. Another challenge when working with molecular similarity are so-called **activity cliffs**, i.e. a small change in a functional group of a molecule may initiate a jump in bioactivity. 

# Quiz

In [ ]:
run_yaml_quiz(f"../corrections/quiz/T4_mol_similarity.yaml", "theoric-quiz")

In [ ]:
run_yaml_quiz(f"../corrections/quiz/T4_mol_similarity.yaml", "code-quiz")